In [2]:
from formatter import YOLOFormatter
from ultralytics import YOLO
import random
import shutil

class YOLODetector:
    def __init__(self, yolo_type="yolov8s.pt"):
        self.yolo_formatter = YOLOFormatter()
        self.yolo_type = yolo_type
        self.model = YOLO(yolo_type)
    
    def format_to_yolo_directories(self, images_source_directory, labels_source_directory):
        images_target_directory = "./datasets/yolo-maritime-flags-dataset/images/train/"
        labels_target_directory = "./datasets/yolo-maritime-flags-dataset/labels/train/"
        self.yolo_formatter.delete_all_files_and_directories(images_target_directory)
        self.yolo_formatter.delete_all_files_and_directories(labels_target_directory)
        self.yolo_formatter.copy(images_source_directory, images_target_directory)
        self.yolo_formatter.copy(labels_source_directory, labels_target_directory)
    
    def create_validation_set(self, images_source_directory, labels_source_directory, number_of_images_per_class):
        images_target_directory = "./datasets/yolo-maritime-flags-dataset/images/val/"
        labels_target_directory = "./datasets/yolo-maritime-flags-dataset/labels/val/"
        if os.path.exists(images_target_directory):
            self.yolo_formatter.delete_all_files_and_directories(images_target_directory)
        if os.path.exists(labels_target_directory):
            self.yolo_formatter.delete_all_files_and_directories(labels_target_directory)
        os.makedirs(images_target_directory)
        os.makedirs(labels_target_directory)
        categories = os.listdir(images_source_directory)
        for category in categories:
            path_to_category = f"{images_source_directory}/{category}"
            number_of_images = len(os.listdir(path_to_category)) # 400
            lower_bound = 10
            upper_bound = number_of_images
            selected_images = [f"{category}_{random.randint(lower_bound, upper_bound)}.jpg" for _ in range(number_of_images_per_class)]
            for image in selected_images:
                source=f"{path_to_category}/{image}"
                destination=f"{images_target_directory}/{image}"
                shutil.copy(source, destination)
                label=str(image).split(".")[0]+".txt"
                path_to_source_label=f"{labels_source_directory}/{category}/{label}"
                path_to_output_label=f"{labels_target_directory}/{label}"
                shutil.copy(path_to_source_label, path_to_output_label)

    def create_test_set(self,images_source_directory, labels_source_directory, number_of_images_per_class):
        images_target_directory = "./datasets/yolo-maritime-flags-dataset/images/test/"
        labels_target_directory = "./datasets/yolo-maritime-flags-dataset/labels/test/"
        if os.path.exists(images_target_directory):
            self.yolo_formatter.delete_all_files_and_directories(images_target_directory)
        if os.path.exists(labels_target_directory):
            self.yolo_formatter.delete_all_files_and_directories(labels_target_directory)
        os.makedirs(images_target_directory)
        os.makedirs(labels_target_directory)
        categories = os.listdir(images_source_directory)
        for category in categories:
            path_to_category = f"{images_source_directory}/{category}"
            number_of_images = len(os.listdir(path_to_category)) # 400
            lower_bound = 10
            upper_bound = number_of_images
            selected_images = [f"{category}_{random.randint(lower_bound, upper_bound)}.jpg" for _ in range(number_of_images_per_class)]
            for image in selected_images:
                source=f"{path_to_category}/{image}"
                destination=f"{images_target_directory}/{image}"
                shutil.copy(source, destination)
                label=str(image).split(".")[0]+".txt"
                path_to_source_label=f"{labels_source_directory}/{category}/{label}"
                path_to_output_label=f"{labels_target_directory}/{label}"
                shutil.copy(path_to_source_label, path_to_output_label)
    
    def fit(self, configuration_file, image_size, number_of_epochs):
        self.model.train(data=configuration_file, imgsz=image_size, epochs=number_of_epochs)
        
    def predict(self, path_to_image):
        self.model(path_to_image, save=True, save_txt=True)

In [3]:
method="SMOTE"
yolo_type="yolov8s.pt"
path_to_test_image=f"./maritime-flags-dataset/{method}_balanced_flags/images/A/A_01.jpg"
images_source_directory=f"./maritime-flags-dataset/{method}_balanced_flags/images/"
labels_source_directory=f"./maritime-flags-dataset/{method}_balanced_flags/labels/"
yolo_configuration_file=f"./data.yaml"
validation_number_of_images_per_class=32
test_number_of_images_per_class=20
image_size=128
number_of_epochs=10

yolo_detector = YOLODetector(yolo_type=yolo_type)
"""
yolo_detector.format_to_yolo_directories(
    images_source_directory=images_source_directory, 
    labels_source_directory=labels_source_directory
)
yolo_detector.create_validation_set(
    images_source_directory=images_source_directory, 
    labels_source_directory=labels_source_directory, 
    number_of_images_per_class=validation_number_of_images_per_class
)
yolo_detector.create_test_set(
    images_source_directory=images_source_directory, 
    labels_source_directory=labels_source_directory,
    number_of_images_per_class=test_number_of_images_per_class
)
"""
yolo_detector.fit(
    image_size=image_size, 
    number_of_epochs=number_of_epochs,
    configuration_file=yolo_configuration_file
)
yolo_detector.predict(
    path_to_image=path_to_test_image
)

New https://pypi.org/project/ultralytics/8.3.44 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.43  Python-3.11.6 torch-2.1.1+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=./data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning D:\Github-Imbalanced\Current-\Imbalanced-Data-Problem-for-Image-Detection\datasets\yolo-maritime-flags-dataset\labels\test.cache... 511 images, 0 backgrounds, 0 corrupt: 100%|██████████| 511/511 [00:00<?, ?it/s]
val: Scanning D:\Github-Imbalanced\Current-\Imbalanced-Data-Problem-for-Image-Detection\datasets\yolo-maritime-flags-dataset\labels\test.cache... 511 images, 0 backgrounds, 0 corrupt: 100%|██████████| 511/511 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 


c:\Users\PanCh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


optimizer: AdamW(lr=0.000333, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 128 train, 128 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10         0G     0.1544     0.7926     0.9261         15        128: 100%|██████████| 32/32 [00:16<00:00,  1.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:08<00:00,  1.88it/s]

                   all        511        511      0.876      0.849      0.927      0.912



10 epochs completed in 0.076 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.43  Python-3.11.6 torch-2.1.1+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 168 layers, 11,135,646 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:07<00:00,  2.02it/s]


                   all        511        511      0.879      0.846      0.927      0.912
                     A         20         20      0.822          1      0.995      0.995
                     B         20         20      0.734          1      0.983      0.961
                     C         20         20      0.918        0.7      0.894      0.886
                     D         20         20      0.926       0.35      0.651      0.651
                     E         20         20      0.784       0.45      0.732      0.726
                     F         20         20      0.966          1      0.995      0.995
                     G         20         20      0.649        0.9      0.883      0.883
                     H         19         19          1      0.311      0.668      0.664
                     I         19         19      0.867          1      0.995      0.995
                     J         20         20          1      0.598      0.965      0.965
                     

In [ ]:
yolo_detector.predict(
    path_to_image=path_to_test_image
)


image 1/1 d:\Github-Imbalanced\Current-\Imbalanced-Data-Problem-for-Image-Detection\maritime-flags-dataset\SMOTE_balanced_flags\images\A\A_01.jpg: 128x128 2 As, 38.0ms
Speed: 0.6ms preprocess, 38.0ms inference, 0.6ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs\detect\train4
1 label saved to runs\detect\train4\labels


In [7]:
results = yolo_detector.model("./combined-maritime-flags/combined_01.jpg", save=True, save_txt=True,  iou=1.0, conf=0.001, classes=None)
print(results)


image 1/1 d:\Github-Imbalanced\Current-\Imbalanced-Data-Problem-for-Image-Detection\combined-maritime-flags\combined_01.jpg: 128x128 1 H, 2 Ls, 21 Ns, 1 P, 17 Rs, 2 Xs, 37.1ms
Speed: 1.0ms preprocess, 37.1ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs\detect\train5
1 label saved to runs\detect\train5\labels
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}
obb: None
orig_img: array([[[242, 233, 236],
        [243, 235, 235],
        [239, 234, 225],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[244, 235, 238],
        [245, 237, 237],
        [242, 237, 228],
        ...,
      